# **Building Custom Named Entity Recognition Model Using Spacy**

https://newscatcherapi.com/blog/train-custom-named-entity-recognition-ner-model-with-spacy-v3

## **Introduction**

In this notebook we are going to adapt a name entity recognition model to our problem of detecting negations, uncertainties and their corresponding scopes. The data we have consists in 319 medical texts with some negations and uncertainties with scopes (or not) and the corresponing indices of start and end of each on of them in the text.

For this task we are going to use the spacy library an the corpus that it provides.

## Spacy library and examples of NER (english)

In [ ]:
import spacy

In the following you will see some examples of name entity recognition using the english corpus en_core_web_lg from spacy and its pretrained model for NER.

In [ ]:
# !python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_lg

2023-05-26 08:08:50.799829: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
# nlp is the model that performs name entity recgnition
nlp = spacy.load("en_core_web_lg")
print(type(nlp))

<class 'spacy.lang.en.English'>


In [ ]:
# nlp receives a text a returns a document object that contains the text and the entities of the text that are of type span object
doc = nlp("Donad Trump was President of USA")

In [ ]:
doc

Donad Trump was President of USA

In [ ]:
type(doc)

spacy.tokens.doc.Doc

In [ ]:
doc.ents

(Donad Trump, USA)

In [ ]:
print(doc.ents[0], type(doc.ents[0]))

Donad Trump <class 'spacy.tokens.span.Span'>


We use the displacy library to show the text process with the entities recognized in colors with the corresponding label

In [ ]:
from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)

## Loading the data and preparing it for the model

In the following we load the data and save it in a DataFrame. The whole data is in a json file but for now we take only the texts with the id.

In [ ]:
import json
import pandas as pd
 
with open('/content/negacio_uab_revised_version.json', 'r') as f:
    json_data = json.load(f)

In [ ]:
data = []
for i in range(len(json_data)):

  id = json_data[i]['data']['id']
  text = json_data[i]['data']['text']
  dict_patient = {'id': id, 'text': text}
  data.append(dict_patient)
  

In [ ]:
data = pd.DataFrame(data)

In [ ]:
data.head()

,id,text
0,18796742,nº historia clinica: ** *** *** nºepisodi: **...
1,18819888,nº historia clinica: ** *** *** nºepisodi: **...
2,18861858,nº historia clinica: ** *** *** nºepisodi: **...
3,18847417,nº historia clinica: ** *** *** nºepisodi: **...
4,18868762,nº historia clinica: ** *** *** nºepisodi: **...


For now we have the texts and the id's. We tried the NER model we just mentioned before with this text to see what happend and you will see that it doesn't work because the language does not match and it doesn't know anything about the negation and uncertainty problem.

In [ ]:
# entities recognized by the model:
doc = nlp(data['text'][0])
doc.ents

(dona data de naixement,
 06.06.1938,
 79,
 procedencia domicil/res.soc servei psiquiatria data d'ingres 10.05.2018,
 d'hospitalitzacio motiu d'ingres paciente,
 79,
 años que,
 acude derivada,
 tratamiento farmaoclogico,
 tratamiento farmacologico,
 tratamiento con venlafaxina y sertralina,
 primera vinculacion con red de salud mental,
 el 2013,
 raiz de la muerte desu marido,
 con sentimientos de tristeza,
 soledad,
 a su marido y su muerte e importanteclinica ansiosa,
 se vincula,
 a csma sant andreu donde inician tratamiento,
 farmacologico con,
 paroxetina sinseguimiento,
 del alta de csma de zona de febrero 2016,
 de alta debido,
 mejoria,
 del t. depresivo,
 tratamiento al alta de sertralina,
 deriva,
 seguimiento,
 -acudio,
 abril de 2018,
 medicamentosa,
 venlafaxina,
 lorazepam, valsrtan/hidroclorotiazida,
 flumazenilo,
 ssf,
 viuda,
 sin hijos,
 tiene un hermano con el que tiene contacto,
 barcelona,
 vive,
 apartamentos,
 mutuam collserola,
 deterioro cognitivo leve-moderad

In [ ]:
# Now in a more visual way, with the entity type printed.
displacy.render(doc, style="ent", jupyter=True)

In [ ]:
# Now we tried with a medical text in englis and you will see that it recognizes very well the entities.
doc = nlp('Dr. John Anderson, a renowned cardiologist from Stanford Medical Center, suggested that patients with high cholesterol should modify their diet to include more heart-healthy foods. In a study published in the New England Journal of Medicine, he found that incorporating foods rich in omega-3 fatty acids, such as salmon and walnuts, can significantly lower the risk of heart disease. Dr. Anderson also advised patients to limit intake of trans fats, which are often found in processed foods. Furthermore, he suggested regular exercise and recommended patients to track their blood pressure using devices approved by the FDA. His research has been endorsed by the American Heart Association, paving a new direction in cardiology.')

In [ ]:
displacy.render(doc, style="ent", jupyter=True)

The objective now is to fine-tune the model so that it recognizes the entities that we want, which are negations, uncertainties and the scopes. For this process we need to prepare the data for the model for the fine-tuning training.

In [ ]:
# We add the new colum that contains the labels of the text with the indices of start and end to the dataframe
labels = []

for i, _ in enumerate(data['id']):
    labels_per_text = []
    for item in json_data[i]['predictions'][0]['result']:

        # For each item we add a tuple with the start and end indices of the item in the text and it is added to the labels list with the corresponding label
        labels_per_text.append((item['value']['start'], item['value']['end'], item['value']['labels'][0])) # tuple: (start, end, label)

    labels.append(labels_per_text)

In [ ]:
# This line adds the new column to the dataframe
data['labels'] = labels

In [ ]:
# Some labels in the first text
data['labels'][0]

[(1350, 1354, 'NEG'),
 (1996, 1999, 'NEG'),
 (2385, 2389, 'NEG'),
 (2490, 2493, 'NEG'),
 (2557, 2560, 'NEG'),
 (2657, 2660, 'NEG'),
 (2748, 2751, 'NEG'),
 (3038, 3049, 'UNC'),
 (3049, 3080, 'USCO'),
 (1354, 1359, 'NSCO'),
 (414, 417, 'NEG'),
 (418, 450, 'NSCO'),
 (2389, 2433, 'NSCO'),
 (2493, 2555, 'NSCO'),
 (2560, 2610, 'NSCO'),
 (2660, 2746, 'NSCO'),
 (2751, 2773, 'NSCO'),
 (1999, 2036, 'NSCO')]

In [ ]:
data.head()

,id,text,labels
0,18796742,nº historia clinica: ** *** *** nºepisodi: **...,"[(1350, 1354, NEG), (1996, 1999, NEG), (2385, ..."
1,18819888,nº historia clinica: ** *** *** nºepisodi: **...,"[(988, 992, NEG), (992, 999, NSCO), (1040, 104..."
2,18861858,nº historia clinica: ** *** *** nºepisodi: **...,"[(896, 899, NEG), (899, 931, NSCO), (1259, 128..."
3,18847417,nº historia clinica: ** *** *** nºepisodi: **...,"[(391, 394, NEG), (394, 428, NSCO), (444, 447,..."
4,18868762,nº historia clinica: ** *** *** nºepisodi: **...,"[(357, 361, NEG), (361, 414, NSCO), (903, 907,..."


This is the spanish language model that we are going to use to train the model:

In [ ]:
!python -m spacy download es_core_news_lg

2023-05-30 18:48:19.522559: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 823.8 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')


In [ ]:
nlp = spacy.load('es_core_news_lg')
nlp

## Training and validation split:

In this part split the data into train and validation sets

In [ ]:
training_data = []
for text, labels_list in zip(data['text'], data['labels']):
  temp_dict = {}
  temp_dict['text'] = text
  temp_dict['entities'] = []
  for label in labels_list:
    start = label[0]
    end = label[1]
    label = label[2].upper()
    temp_dict['entities'].append((start, end, label))
  training_data.append(temp_dict)
  
print(training_data[0])

{'text': " nº historia clinica: ** *** *** nºepisodi: ******** sexe: dona data de naixement: 06.06.1938 edat: 79 anys procedencia domicil/res.soc servei psiquiatria data d'ingres 10.05.2018 data d'alta 10.05.2018 16:46:41 ates per ****************, ************; *************, assumpta informe d'alta d'hospitalitzacio motiu d'ingres paciente de 79 años que acude derivada a urgencias de psiquiatria tras sim. antecedents -sin alergias mediamentosas conocidas - hipertension arterial en tratamiento farmaoclogico con tres farmacos. - dislipemia en tratamiento farmacologico. - sindrome ansioso depresivo de larga evolucion actualmente en tratamiento con venlafaxina y sertralina. refiere primera vinculacion con red de salud mental en el 2013 a raiz de la muerte desu marido, con sentimientos de tristeza, soledad, rumiaciones respecto a su marido y su muerte e importanteclinica ansiosa. se vincula a csma sant andreu donde inician tratamiento farmacologico con paroxetina sinseguimiento posterior.

In [ ]:
len(training_data)

319

In [ ]:
validation_data = training_data[300:]
training_data = training_data[: len(training_data) - len(validation_data)]

In [ ]:
print(f'training set has {len(training_data)} texts and the validation set has {len(validation_data)} texts')

training set has 300 texts and the validation set has 19 texts


In [ ]:
training_data[0]['text']

" nº historia clinica: ** *** *** nºepisodi: ******** sexe: dona data de naixement: 06.06.1938 edat: 79 anys procedencia domicil/res.soc servei psiquiatria data d'ingres 10.05.2018 data d'alta 10.05.2018 16:46:41 ates per ****************, ************; *************, assumpta informe d'alta d'hospitalitzacio motiu d'ingres paciente de 79 años que acude derivada a urgencias de psiquiatria tras sim. antecedents -sin alergias mediamentosas conocidas - hipertension arterial en tratamiento farmaoclogico con tres farmacos. - dislipemia en tratamiento farmacologico. - sindrome ansioso depresivo de larga evolucion actualmente en tratamiento con venlafaxina y sertralina. refiere primera vinculacion con red de salud mental en el 2013 a raiz de la muerte desu marido, con sentimientos de tristeza, soledad, rumiaciones respecto a su marido y su muerte e importanteclinica ansiosa. se vincula a csma sant andreu donde inician tratamiento farmacologico con paroxetina sinseguimiento posterior. consta i

In [ ]:
training_data[0]['entities']

[(1350, 1354, 'NEG'),
 (1996, 1999, 'NEG'),
 (2385, 2389, 'NEG'),
 (2490, 2493, 'NEG'),
 (2557, 2560, 'NEG'),
 (2657, 2660, 'NEG'),
 (2748, 2751, 'NEG'),
 (3038, 3049, 'UNC'),
 (3049, 3080, 'USCO'),
 (1354, 1359, 'NSCO'),
 (414, 417, 'NEG'),
 (418, 450, 'NSCO'),
 (2389, 2433, 'NSCO'),
 (2493, 2555, 'NSCO'),
 (2560, 2610, 'NSCO'),
 (2660, 2746, 'NSCO'),
 (2751, 2773, 'NSCO'),
 (1999, 2036, 'NSCO')]

In [ ]:
training_data[0]['text'][1350:1354] # the first  word labeled in the first text

'sin '

## Serialized format:

Now we transform the data into a serialized format (.spacy) that can be used for training a Named Entity Recognition (NER) model in SpaCy.

In [ ]:
# Import the DocBin class for creating binary data from "Doc" objects
from spacy.tokens import DocBin
# Import tqdm for creating progress bars
from tqdm import tqdm

# Create a DocBin object for storing Doc objects
doc_bin = DocBin()

In [ ]:
# Import the function for filtering overlapping spans
from spacy.util import filter_spans


# Iterate over each example in the training data
for training_example  in tqdm(training_data):

    text = training_example['text']

    # Extract the entity labels from the current example
    labels = training_example['entities']

    # Create a Doc object from the text
    doc = nlp.make_doc(text) 

    # Create an empty list to store the entity spans
    ents = []

    # Iterate over each entity label
    for start, end, label in labels:
        # Create a Span object for the current entity
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        
        # If a span could not be created, print a warning and skip this entity
        if span is None:
            print("Skipping entity")
        else:
            # If a span was successfully created, add it to the list of spans
            ents.append(span)

    # Filter out overlapping spans
    filtered_ents = filter_spans(ents)
    
    # Assign the list of non-overlapping spans to the "ents" attribute of the Doc object
    doc.ents = filtered_ents 
    
    # Add the Doc object to the DocBin
    doc_bin.add(doc)

# Write the DocBin to disk in binary format
doc_bin.to_disk("train.spacy") 


  1%|▏         | 4/300 [00:00<00:07, 37.67it/s]

Skipping entity


  6%|▋         | 19/300 [00:00<00:07, 38.40it/s]

Skipping entity


 14%|█▍        | 43/300 [00:00<00:04, 58.14it/s]

...

 97%|█████████▋| 290/300 [00:05<00:00, 42.92it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity


 98%|█████████▊| 295/300 [00:05<00:00, 35.72it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


100%|██████████| 300/300 [00:05<00:00, 51.00it/s]


In [ ]:
# Iterate over each example in the validation data
for val_example  in tqdm(validation_data):
    text = val_example['text']

    # Extract the entity labels from the current example
    labels = val_example['entities']

    # Create a Doc object from the text
    doc = nlp.make_doc(text) 

    # Create an empty list to store the entity spans
    ents = []

    # Iterate over each entity label
    for start, end, label in labels:
        # Create a Span object for the current entity
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        
        # If a span could not be created, print a warning and skip this entity
        if span is None:
            print("Skipping entity")
        else:
            # If a span was successfully created, add it to the list of spans
            ents.append(span)

    # Filter out overlapping spans
    filtered_ents = filter_spans(ents)
    
    # Assign the list of non-overlapping spans to the "ents" attribute of the Doc object
    doc.ents = filtered_ents 
    
    # Add the Doc object to the DocBin
    doc_bin.add(doc)

# Write the DocBin to disk in binary format
doc_bin.to_disk("validation.spacy") 


 42%|████▏     | 8/19 [00:00<00:00, 32.84it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


100%|██████████| 19/19 [00:00<00:00, 37.28it/s]


Skipping entity
Skipping entity
Skipping entity


## Training the model

For training the model we need a configuration file that defines the training configuration, batch size, number of epochs and other options.

The following is a link to generate the configuration file to create and train the model:

If the model is already trained and saved, there is no need to run these lines, just load the model in the next section.

In [ ]:
# https://spacy.io/usage/training#quickstart

In [ ]:
# Here is the initialization of the file of configurations
!python -m spacy init fill-config base_config.cfg config.cfg

2023-05-24 16:15:41.757885: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
# This line starts the training of the model with the configurations given and the train set and validation set given.
# The model is saved periodically to maintain the model with the best results.
!python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./validation.spacy 

2023-05-30 19:41:54.451807: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2023-05-30 19:41:59,188] [INFO] Set up nlp object from config
[2023-05-30 19:41:59,207] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-05-30 19:41:59,212] [INFO] Created vocabulary
[2023-05-30 19:42:05,343] [INFO] Added vectors: es_core_news_lg
[2023-05-30 19:42:08,437] [INFO] Finished initializing nlp object
[2023-05-30 19:42:37,117] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    403.50    0.31    0.19    0.79    0

## Loading the model and predicting entities

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Here the model is loaded and is ready to be tested
nlp_ner = spacy.load("/content/drive/MyDrive/nlp_project/model-last/model-best") # insert the path of the folder where the model is saved. 

In [ ]:
import random # we use random to do random selections of text

Here we give the model some texts from the data to predict the entities:

In [ ]:
# first we tried using random texts from the whole data
doc = nlp_ner(data['text'][random.choice(data.index)])

colors = {"NEG": "#FF0000", "NSCO": "#FFA500", "UNC": "#800080", "USCO": "#FFC0CB"}
options = {"colors": colors}

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

In [ ]:
# We also tried using texts form the validation set:
doc = nlp_ner(validation_data[random.randint(0, len(validation_data))]['text'])

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

### Testing with texts that are not from the data (medical texts from the internet)

In [ ]:
text = '''nº historia clinica: ** *** *** nºepisodi: ******** sexe: home data de naixement: 27.07.1956 edat: 67 anys procedencia domicil/res.soc servei cirurgia general data d'ingres 15.05.2023 data d'alta 28.05.2023 11:20:22 ates per **************, *****; ***************, ************; ************, ********** informe d'alta d'hospitalitzacio motiu d'ingres dolor abdominal agudo y nausea post colecistectomia.

antecedents sin alergias medicamentosas conocidas - hipertension arterial en tratamiento con losartan. - enfermedad pulmonar obstructiva cronica. - colecistectomia laparoscopica 05/05/2023 en huvh.

proces actual hombre de 67 años intervenido de colecistectomia laparoscopica el dia 05/05/2023 por litiasis biliar, presentando cuadro de dolor abdominal y nausea sin vomitos. fiebre. Se sospecha que el paciente pueda sufrir de fuerte diarrea explosiva.

exploracio fisica buen estado general, afebril, hdm estable. abdomen: herida quirurgica con buen aspecto, levemente distendido, no doloroso a la palpacion, sin signos de irritacion peritoneal.

exploracio complementaria ag: hb 15, leu 8.25, plaq 265, inr 1.03, glu 122, cr 1.22, na 133.5, pcr 10.'''

In [ ]:
doc = nlp_ner(text)

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

In [ ]:
text = '''Número de historia clínica: ** *** *** Número de episodio: ******** Sexo: masculino Fecha de nacimiento: 14.03.1953 Edad: 66 años Procedencia: domicilio/ res.soc. Servicio de oncología médica Fecha de ingreso: 25.03.2019 Fecha de alta: 02.04.2019 Atendido por: ************, *****; *******************, *****************

Informe de alta hospitalaria:

Motivo de ingreso: Paciente masculino de 66 años que acude a urgencias por vómitos y mareos. No se puede descartar ni confirmar alergias a medicamentos. Historia oncológica: paciente con glioblastoma medular. Resección parcial realizada. Se optó por esquema de qt -rdt stupp tmz y rdt hasta dt 60 gy rt /qt stupp temozolomida.

Tras el primer ciclo de tmz en 6/4/18, se realiza resonancia magnética cada 3 ciclos. Se dan un total de 7 ciclos con buena tolerancia. Posible progresión de la enfermedad a los 7 ciclos en forma de diseminación medular. No se puede confirmar ni descartar signos significativos clínicamente. Se decide reirradiación con tmz como radiosensibilizador.

El paciente sufrió un evento de trombosis venosa profunda en las extremidades inferiores bilateral proximal en abril/18. Eco-doppler realizado en 24.07.18: venas femorales permeables compresibles con fascicidad respiratoria, venas poplíteas y gemelares permeables compresibles sin trombo identificado.

En septiembre/18, con la trombosis venosa profunda resuelta y finalizada la quimioterapia, se pasa a profilaxis secundaria. En octubre 18 se realiza radioterapia/quimioterapia sobre los focos de progresión y temozolomida concomitante.

En el seguimiento con resonancia magnética en enero 2019, se confirman signos radiológicos de progresión tumoral por discreto crecimiento de los implantes leptomeningeos dorsales ya conocidos. Se identifican signos de carcinomatosis cranail, que no se encontraban en el estudio de junio 2018.

El paciente está actualmente bajo tratamiento de segunda línea con bevacizumab en monoterapia. El estudio de resonancia magnética espinal de control en 06.03.19 muestra mejora radiológica de la enfermedad dada por una discreta disminución de la lesión a nivel del cono, así como significativa disminución de los implantes a nivel dorsal alto y posible resolución del pequeño implante a nivel cervical.

El paciente es seguido en consulta de oncología por la Dra. Vieito / Dra. Gonzalez. Durante su ingreso hospitalario, el paciente ha mantenido hemodinámicamente estable y afebril.

Se decide dar de alta estableciendo normas de reconsulta y medidas terapéuticas. El paciente iniciará nueva medicación y continuará con su medicación habitual. Se realizará un seguimiento en consulta externa de oncología según citación. Se explican signos de alarma y reconsulta.

Este informe pretende ser un resumen de los eventos médicos significativos del paciente to: ChatGPT Assistant<|im_sep|>Paciente: Sexo masculino, Fecha de nacimiento: 14.03.1953, Edad: 66 años. Admitido en el departamento de oncología médica el 25.03.2019 y dado de alta el 02.04.2019. Informe de alta proporcionado por el equipo de oncología.

El paciente, un hombre de 66 años, acudió a emergencias con vómitos y mareos. No se confirmó ni descartó la presencia de alergias a medicamentos. El paciente tiene un historial de glioblastoma medular y se le practicó una resección parcial.

Tras la cirugía, se decidió continuar con quimioterapia y radioterapia (esquema Stupp), utilizando Temozolomida (TMZ). Se completaron 7 ciclos de tratamiento, los cuales fueron bien tolerados. Se sospecha de progresión de la enfermedad en la forma de diseminación medular, aunque esta no se ha confirmado ni descartado clínicamente. Se decide realizar una segunda irradiación con TMZ concomitante como radiosensibilizador.

El paciente experimentó una trombosis venosa profunda (TVP) en las extremidades inferiores en abril de 2018. A partir de septiembre de 2018, y tras la resolución de la TVP, se inició una profilaxis secundaria. Se detectaron signos de carcinomatosis craneal, que no se habían confirmado ni descartado previamente.

Actualmente, el paciente está en su segunda línea de tratamiento con bevacizumab en monoterapia. Una RM espinal realizada el 06.03.19 sugiere una mejora radiológica de la enfermedad. Sin embargo, se advierten cambios en la señal de la médula ósea en la región dorsal y adelgazamiento del cordón espinal dorsal, hallazgos atribuibles a cambios post-radiación. No se observan lesiones nuevas, pero la posibilidad no puede confirmarse ni descartarse.

El paciente ha estado sintiendo inestabilidad cefálica y sensaciones de vértigo, que se exacerban con el movimiento de la cabeza. Se le ha prescrito betahistina, pero el aumento de la dosis no ha mostrado ninguna mejora confirmada ni descartada. Se realizó una TC craneal el 21/03 que evidenció signos sugestivos de carcinomatosis leptomeningea, aunque no se han descartado procesos agudos.

Durante su hospitalización, el paciente ha permanecido hemodinámicamente estable y sin fiebre. Se decidió darle de alta con indicaciones para reconsulta y medidas terapéuticas. El diagnóstico principal es R42, mareo y vértigo. Se le recetó omeprazol y dexametasona, además de su medicación habitual, y se le dio un número de teléfono para atención inmediata en caso de cualquier signo de alarma.
'''

In [ ]:
doc = nlp_ner(text)

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

In [ ]:
text = '''Número de historia clínica: ** *** *** Número de episodio: ******** Sexo: masculino Fecha de nacimiento: 03.11.1956 Edad: 66 años Procedencia: domicilio/ res.soc. Servicio de oncología médica Fecha de ingreso: 18.03.2019 Fecha de alta: 27.03.2019 Atendido por: ************, *****; *******************, *****************

Informe de alta hospitalaria:

Motivo de ingreso: Paciente masculino de 66 años que acude a urgencias por pérdida de equilibrio y dolores de cabeza. No se puede descartar ni confirmar alergias a medicamentos. Historia oncológica: paciente con glioma de alto grado. Se realizó una resección parcial del tumor y se inició quimioterapia con un esquema de qt -rdt stupp tmz y rdt hasta dt 60 gy rt /qt stupp temozolomida.

Después del primer ciclo de temozolomida en 5/5/18, se realizó una resonancia magnética cada tres ciclos. Se completaron un total de 7 ciclos con buena tolerancia. Se sospecha de progresión de la enfermedad después de los 7 ciclos en forma de diseminación medular. No se puede confirmar ni descartar signos clínicos significativos. Se decide una reirradiación con temozolomida como radiosensibilizador.

El paciente tuvo un evento de trombosis venosa profunda en las extremidades inferiores bilaterales proximales en mayo/18. Un eco-doppler realizado en 24.08.18 confirmó venas femorales permeables y compresibles con fascicidad respiratoria, venas poplíteas y gemelares permeables y compresibles sin trombo identificado.

En octubre/18, después de la resolución de la trombosis venosa profunda y la finalización de la quimioterapia, se comenzó una profilaxis secundaria. En noviembre 18, se inició radioterapia/quimioterapia sobre los focos de progresión y temozolomida concomitante.

En la resonancia magnética de seguimiento en diciembre 2018, se confirmaron signos radiológicos de progresión tumoral con crecimiento discreto de los implantes leptomeningeos dorsales conocidos. Se identificaron signos de carcinomatosis craneal que no estaban presentes en el estudio de julio 2018.

Actualmente, el paciente está bajo tratamiento de segunda línea con bevacizumab en monoterapia. La resonancia magnética espinal de control realizada en 07.03.19 muestra una mejora radiológica de la enfermedad con una disminución discreta de la lesión a nivel del cono, así como una disminución significativa de los implantes a nivel dorsal alto y posible resolución del pequeño implante a nivel cervical.

El paciente es seguido en consulta de oncología por la Dra. Vieito / Dra. Gonzalez. Durante su ingreso hospitalario, el paciente ha estado hemodinámicamente estable y afebril.

Se decide dar de alta estableciendo normas de reconsulta y medidas terapéuticas. El paciente iniciará nueva medicación y continuará con su medicación habitual. Se realizará un seguimiento en consulta externa de oncología según cit.'''

In [ ]:
spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

In [ ]:
doc = nlp_ner('el paciente ha sido diagnosticado con probable infección, pero sintomas de demencia negativos.')

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

## Testing with texts that are not from the data and are not related to medical texts:

Testing with a legal texts:

In [ ]:
text = '''El acuerdo puede no considerarse completamente ejecutable a menos que todas las partes involucradas hayan dado su consentimiento de manera inequívoca. Sin embargo, el Sr. Thompson ha expresado incertidumbre con respecto a su decisión de unirse a la sociedad. No puede ser obligado a unirse al acuerdo si existe la más mínima duda sobre su legalidad o equidad. No está inmediatamente claro si las reclamaciones por incumplimiento de contrato serán aceptadas en el tribunal, dada la ambigüedad en el lenguaje de la cláusula 4. El tribunal se reserva el derecho de no aplicar ninguna sanción si el acusado desconocía la infracción. Mientras el documento podría potencialmente vincular a las partes, existe incertidumbre con respecto a su efectividad a la luz de una posible no divulgación por cualquiera de las partes. Cualquier enmienda al acuerdo no se considerará válida sin el consentimiento expreso por escrito de todas las partes involucradas. Es dudoso si la cláusula de no competencia del acuerdo puede hacerse cumplir en diferentes jurisdicciones. El demandante no puede perseguir daños si no se demuestra que el acusado fue negligente.'''

In [ ]:
doc = nlp_ner(text)
colors = {"NEG": "#FF0000", "NSCO": "#FFA500", "UNC": "#800080", "USCO": "#FFC0CB"}
options = {"colors": colors}

In [ ]:
spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

In [ ]:
text = '''El contrato jamás será considerado totalmente vinculante a menos que todas las partes implicadas hayan manifestado su consentimiento de forma irrefutable. Aun así, el Señor García ha mostrado dudas respecto a su decisión de sumarse a la sociedad. No existe obligación alguna para García de unirse al pacto si persisten dudas acerca de la justicia o la legalidad del mismo. No queda del todo claro si las reclamaciones por quebrantamiento del contrato se mantendrán en pie en el tribunal, debido a la ambigüedad de la cláusula 4. El tribunal se reserva la potestad de desestimar cualquier sanción si el acusado desconocía la transgresión. Aunque el documento podría ligar a las partes en teoría, persiste la incertidumbre sobre su efectividad en caso de omisión de información por cualquiera de las partes. Cualquier cambio al contrato será inválido a menos que cuente con el consentimiento por escrito de todas las partes implicadas. Se cuestiona si la cláusula de no competencia del contrato puede ser impuesta en diferentes jurisdicciones. La parte demandante carecerá de derecho a reclamar daños y perjuicios si no se prueba la negligencia del acusado.'''

In [ ]:
doc = nlp_ner(text)
spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

In [ ]:
text = '''La transferencia de la propiedad nunca se llevará a cabo a menos que todas las partes hayan otorgado su consentimiento de manera indiscutible. Pero el Señor Pérez ha mostrado reservas en cuanto a su intención de adquirir el inmueble. No existe la menor obligación para Pérez de proceder con la compra si aún hay interrogantes sobre la legitimidad o la equidad del contrato. No es completamente evidente si las demandas por incumplimiento del contrato tendrán validez en el tribunal, considerando el lenguaje ambiguo en la cláusula 7. El tribunal tiene la prerrogativa de rechazar cualquier penalización si el demandado ignoraba la infracción. Aunque el documento puede en teoría obligar a las partes, queda en el aire la duda sobre su validez si existiera retención de información por cualquier de las partes. Toda modificación al contrato carecerá de efecto sin el consentimiento expreso y por escrito de todos los implicados. Se plantea la duda de si la cláusula de no competencia del acuerdo podría aplicarse en diversas jurisdicciones. El demandante se encontrará incapacitado para solicitar indemnización si no se logra demostrar el dolo del demandado.'''

In [ ]:
doc = nlp_ner(text)
spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

In [ ]:
text = '''El contrato estipula que todas las partes deben dar su consentimiento de manera indiscutible para la transferencia de la propiedad. El Señor Pérez ha mostrado interés en adquirir el inmueble, pero todavía hay algunas dudas sobre la legitimidad del contrato. Si bien la cláusula 7 es un tanto ambigua, no se espera que esto afecte la validez de las demandas por incumplimiento del contrato. El tribunal tiene la prerrogativa de rechazar cualquier penalización, pero esta situación se ve poco probable dada la evidencia presentada. Aunque se plantea la cuestión de la aplicabilidad de la cláusula de no competencia en diversas jurisdicciones, se espera que este punto sea aclarado en las próximas negociaciones. El demandante se encuentra preparado para solicitar indemnización, a menos que no se logre demostrar el dolo del demandado.'''

In [ ]:
doc = nlp_ner(text)
spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

Testing with a text about sport:

In [ ]:
text = '''El jugador no podrá ser transferido a otro equipo a menos que ambas partes hayan acordado los términos del contrato de manera indiscutible. Sin embargo, hay cierta incertidumbre sobre si el jugador desea realmente dejar su actual club. No puede ser obligado a firmar con el nuevo equipo si hay dudas sobre la igualdad de las condiciones contractuales. No está claro de inmediato si el jugador estará en forma para el comienzo de la próxima temporada, debido a una lesión reciente. La asociación deportiva se reserva el derecho de no sancionar al jugador si no estaba consciente de la infracción de las reglas. Mientras que el contrato podría vincular potencialmente al jugador y al equipo, existe incertidumbre sobre su validez si no se ha revelado por completo el estado de salud del jugador. Cualquier cambio en el contrato no será considerado válido sin el consentimiento expreso por escrito de ambas partes. Existe cierta duda sobre si las cláusulas de no competencia en el contrato pueden hacerse cumplir si el jugador decide retirarse. El equipo no podrá exigir compensación si no se puede demostrar que el jugador ha infringido de forma deliberada las normas del equipo.'''

In [ ]:
doc = nlp_ner(text)
spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

## Conclusions

We can conclude that our model is able to detect negations and uncertainties with the scopes very well most of the times, of course it has some problems in some cases, for example, some times it does not take the full scope of a negation or an uncertainty, some times the scope contains a negation inside so it is cosidered as a negation when it should be the scope of an uncertainty and some times the uncertainty scope takes also the uncertainty part as scope. 

We have test our model with some texts that are not medical and got that it is able to recognize well the negations and uncertainties but most of the scopes look very strange, this could be because the model is not used to this kind of contexts that have no relation with medical texts. 

We thing that the model should be finetuned in order to detect correctly negations, uncertainties and scopes in other specific topics.